# Bayesian Inference with IBIS's MCMC Sampling

In order to better understand our model inputs, Atwood number and velocity, we're going to use Bayesian inference to combine our model data and prior knowledge with observed, experimental data. In this case the posterior distribution that we're after is the distribution of values of mixing width at a certain time, and that distribution has been informed by both the physics model and observed data. Since it can be complicated to compute the posterior distribution we will use IBIS's MCMC sampling module to approximate it. 

## Connect to the Kosh store to access datasets

Again, we can connect to our Kosh store and ensemble to conveniently have access to all the data we need. We're going to need both the simulation data and experimental data for MCMC sampling.

In [ ]:
import numpy as np
import kosh
import scipy.stats as sts
from sklearn.preprocessing import MinMaxScaler as MMS
from sklearn.gaussian_process import GaussianProcessRegressor as GPR
from ibis import mcmc
import os
import matplotlib
matplotlib.use('agg')


# Connect to the Kosh store and read in datasets
store_dir = os.path.join(os.getcwd(), "Experiments/pyranda.sql")
store = kosh.connect(store_dir)

# The experimental data was saved to the ensemble in our Kosh store
experiments_ensemble = next(store.find_ensembles(name="experiments"))
exp_uri = next(experiments_ensemble.find(mime_type="pandas/csv")).uri

# The simulation data was just generated with the workflow manager (Maestro or Merlin)
name = os.listdir(os.path.join(os.getcwd(), "RT_STUDIES"))[0]
sim_ensemble = next(store.find_ensembles(name=name))
sim_uri = next(sim_ensemble.find(mime_type="pandas/csv")).uri

# Use the URI to read in datasets
rt_exp_data = np.genfromtxt(exp_uri, delimiter=',')
rt_sim_data = np.genfromtxt(sim_uri, delimiter=',')

# Separate inputs and outputs for experimental and simulation data
xexp = rt_exp_data[:,:2]
yexp = rt_exp_data[:,2:]
xsim = rt_sim_data[:,:2]
ysim = rt_sim_data[:,2:]

# How many models do we have to evaluate?
# Find the number of outputs
nmodels = ysim.shape[1]

## Train the Surrogate Model

Let's train our Gaussian process model on the simulation data. We want this surrogate model to be able to emulate the real simulation, but provide predictions much faster. 

In [ ]:
# The GP model performs better when the inputs are scaled
scaler = MMS()
scaled_samples = scaler.fit_transform(xsim)

# Build the GP surrogates for the number of models
surrogates = {}
for i in range(nmodels):
    ygp = ysim[:,i]
    name = output_names[i]
    surrogates[name] = GPR().fit(scaled_samples, ygp)

## Defining variables for MCMC sampling

We need to provide a lot of information for the MCMC sampling function. We'll start by defining information for the inputs.

In [ ]:
# Start by instantiating the mcmc object
default_mcmc = mcmc.DefaultMCMC()

# Input information
input_names = ['atwood_num', 'vel_mag']

# Calculate standard deviation for simulation input features
sim_std = np.std(xsim, axis=1)

# We define the min and max for each input
ranges = [[.3, .8], [.7, 1.3] ]

# Add inputs
# We're using uninformative priors for both inputs
for i, name in enumerate(input_names):
    default_mcmc.add_input(name, 0.0, 1.0, sim_std[i], sts.uniform().pdf,
                           unscaled_low=ranges[i][0], unscaled_high=ranges[i][1], scaling='lin')

Next we define information for the outputs.

In [ ]:
# Output information
output_names = []
for i in range(nmodels):
    output_names.append( 'mix_width-%s' % i)

# Calculate experimental mean and standard deviation
# At this time we can only provide one experimental value
expMean = yexp.mean(axis=0)
expStd  = yexp.std(axis=0)

# Add outputs
for name, mean, std in zip(output_names, expMean, expStd):
    if "10" in name:
        default_mcmc.add_output("RTexp", name, surrogates[name], mean , std, input_names )

## Run MCMC Sampling

We will define the rest of the information and run the chains for sampling the posterior distribution.

total: We are going to run chains of "total" in length. That's the total number of samples, but we will need to check the trace plot to see how the sampling looks. 

Start: Start means a starting value for each input. Here we are starting with .5 for both Atwood number and velocity. These are not likely to be the values that product the highest probability for the posterior distribution. So the sampling may explore different local maximums until it finds the global maximum. 

burn: We will want to only keep the samples that are exploring around the global maximum. The inital samples may wander, and we will drop them from the samples we keep. This is called the "burn-in" period. In this case we will drop the first 500 samples.

every: There is some correlation between the samples in the chain. The closer the samples are together, the more correlated they are. If we keep every 2 samples in this case, it reduces the correlation between our samples.

n_chains: We can run multiple independent sampling chains in parallel.

prior_only: Whether to run the chains on just the prior distributions.

seed: The random seed for the Metropolis-Hastings algorithm that chooses the next sample for the chain.

In [ ]:
# Run the MCMC chains to get samples approximating the posterior distribution
default_mcmc.run_chain(total=10000,
                       burn=500,
                       every=2,
                       start={name: .5 for name in input_names},
                       n_chains=3,
                       prior_only=False,
                       seed=15)

chains = default_mcmc.get_chains(flattened=False, scaled=True)

## MCMC Sampling Plots

We'll start by looking at the trace plots. We want to see a good sampling of the space, and since we already removed the "burn-in" samples the center of the mixing should stay in one place.

In [ ]:
for input_n in input_names:
    plot = default_mcmc.trace_plot(input_name=input_n)

Next we will view the histogram showing us a rough approximation of the posterior distribution for the mixing width. 

In [ ]:
outvar = list(default_mcmc.outputs.keys())[0]
post_pp = default_mcmc.posterior_predictive_plot(outvar, bins=10)

Now we can see the more informed distributions for our uncertain parameters.

In [ ]:
for input_n in input_names:
    hist_plot = default_mcmc.histogram_plot(input_name=input_n, bins=25, density=True, alpha=.5)

In the future we could use these distributions for Atwood number and density to make better predictions with error estimates.